In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyro
import torch
import cellij
import numpy as np
from pyro.nn import PyroModule
import pyro.distributions as dist

/Users/martin/miniconda3/envs/cellij/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mdata = cellij._data.Importer().load_CLL()
mdata["mrna"]

AnnData object with n_obs × n_vars = 136 × 5000

In [7]:
model = cellij.core.models.MOFA(n_factors=50, sparsity_prior="Horseshoe")
model.add_data(data=mdata, na_strategy="knn_by_obs")

In [8]:
model.fit(likelihood="Normal",  epochs=1000, verbose_epochs=50)

Epoch      0:     7518824.73
Epoch     50:     5077699.86
